In [1]:
from IPython.core.magic import register_line_cell_magic

## Write a Jupyter Magic

In [2]:
@register_line_cell_magic
def countwords(line, cell=None):
    if cell is None:
        return len(line.split())
    return len(line.split()) + len(cell.split())

In [3]:
%countwords this is a line magic

5

In [4]:
%%countwords

this is a cell magic

5

In [5]:
%%countwords this is a
line and cell magic

7

## Profile the speed of list comprehension vs. for loops

In [11]:
import timeit

In [12]:
list_comp_times = []
for_loop_times = []

for size in [10 ** i for i in range(7)]:
    print(f"Profiling with size {size}")
    time1 = timeit.Timer('[i for i in vec]', setup=f'vec = range({size})').timeit(number=1000)
    time2 = timeit.Timer("""
res=[]
for i in vec:
    res.append(i)
""", setup=f'vec = range({size})').timeit(number=1000)
    list_comp_times.append((size, time1))
    for_loop_times.append((size, time2))

In [31]:
import pandas as pd

pd.DataFrame(list_comp_times, columns=['size', 'list_comp']).merge(
    pd.DataFrame(for_loop_times, columns=['size', 'for_loop']), on='size'
)

,size,list_comp,for_loop
0,1,0.000356,0.000221
1,10,0.000750,0.001318
2,100,0.003998,0.008805
3,1000,0.031837,0.053504
4,10000,0.238424,0.504423
5,100000,3.071976,5.849691
6,1000000,36.193432,65.095156


## Prime numbers

In [35]:
def prime(nums):
    return [num for num in nums if num > 1 and all(num % divisor != 0 for divisor in range(2, int(num ** 0.5) + 1))]

In [36]:
prime(range(100))

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

## Extend the Vector class

In [158]:
class Vector:

    def __init__(self, *nums):
        self.data = nums

    def __repr__(self):
        return f'{self.__class__.__name__}{self.data}'

    def __str__(self):
        return str(self.data)

    def __getitem__(self, index):
        if isinstance(index, int):
            return self.data[index]

        if isinstance(index, slice):
            return Vector(*self.data[index])

        raise TypeError(f"Invalid type of index: {type(index)}")

    def __abs__(self):
        """Euclidean distance"""
        return sum([i ** 2 for i in self.data]) ** 0.5

    def __add__(self, other):
        if not isinstance(other, Vector):
            raise TypeError(f"Cannot add Vector with type {type(other)}")

        if len(self.data) != len(other.data):
            raise ValueError(
                f"Cannot add vectors with different size: {len(self)} vs. {len(other)}")

        return Vector(*[left + right for left, right in zip(self, other)])

    def __mul__(self, scalar):
        return Vector(*(i * scalar for i in self.data))

    def __rmul__(self, scalar):
        """Needed to support scalar * vector"""
        return self * scalar

    def __len__(self):
        return len(self.data)

    def __pow__(self, exponent):  # raise all values of vector to an exponent
        return Vector(*(i ** exponent for i in self.data))

In [159]:
v1 = Vector(1, 2, 3)
v1

Vector(1, 2, 3)

In [160]:
print(v1)

(1, 2, 3)


In [161]:
abs(v1)

3.7416573867739413

In [162]:
v2 = Vector(1)

In [163]:
v1 + v2

ValueError: Cannot add vectors with different size: 3 vs. 1

In [164]:
v2 = Vector(1, 1, 1)
v1 + v2

Vector(2, 3, 4)

In [165]:
v2 * 1.2

Vector(1.2, 1.2, 1.2)

In [166]:
1.2 * v2

Vector(1.2, 1.2, 1.2)

In [167]:
v1[1]

2

In [168]:
v1[:2]

Vector(1, 2)

## Case-insensitive Dictionary

In [320]:
from collections import UserDict


class CaseInsensitiveDict(UserDict):
        
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._keys = {k: k.lower() for k in self.data}

    def __setitem__(self, key, val):
        if key in self:
            del self[key]
        self.data[key] = val
        self._keys[key.lower()] = key

    def __getitem__(self, key):
        return self.data[self._keys[key.lower()]]

    def __delitem__(self, key):
        lower_key = key.lower()
        del self.data[self._keys[lower_key]]
        del self._keys[lower_key]

    def __contains__(self, key):
        return key.lower() in self._keys

    def __iter__(self):
        return iter(self._keys.values())

In [321]:
d = CaseInsensitiveDict()
d['A'] = 3
d['b'] = 4
d

{'A': 3, 'b': 4}

In [322]:
'a' in d

True

In [323]:
'A' in d

True

In [324]:
del d['B']

In [325]:
d

{'A': 3}

In [326]:
d.update({'B': 4, 'a': 5})

In [327]:
d

{'B': 4, 'a': 5}

In [328]:
list(d)

['B', 'a']

In [329]:
list(d.items())

[('B', 4), ('a', 5)]